In [ ]:
import plotly.tools as tls
import cufflinks as cf
import pandas as pd
import numpy as np
import cx_Oracle
import ephem
import matplotlib.pyplot as plt
import plotly.plotly as py
from astroML.plotting import plot_tissot_ellipse
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [ ]:
df = pd.read_csv('data_files/test.txt', sep=' ')

In [ ]:
df.ix[0,1]

In [ ]:
df.head()

In [ ]:
data_group = df.groupby(['ant1name', 'ant2name'])

In [ ]:
data_group.agg({'freq': np.mean, 'chan': np.count_nonzero})

In [ ]:
data_raw = df.groupby(['ant1name', 'ant2name', 'chan']).y.mean()
data_raw.head(30)

In [ ]:
data_raw.unstack().head(20)

In [ ]:
pd.options.display.max_columns = 200
data_raw.unstack().head(20)

In [ ]:
data_raw = data_raw.unstack().reset_index()
data_raw.head()

In [ ]:
data_raw.to_excel('test.xls', index=False)

In [ ]:
todegclean = np.degrees(np.arcsin(np.sin(np.radians(data_raw.iloc[:,2:]))))

In [ ]:
todegclean.head()

In [ ]:
todegclean['mean'] = todegclean.mean(axis=1)

In [ ]:
todegclean.head()

In [ ]:
data_clean = todegclean.iloc[:,:-1].apply(lambda x: x - todegclean.iloc[:,-1])
data_clean.head(20)

In [ ]:
data_ready = pd.merge(data_raw[['ant1name', 'ant2name']], todegclean, left_index=True, right_index=True)
data_ready.head()

## Plot.ly

In [ ]:
data_clean2 = data_clean.unstack().reset_index().copy()

In [ ]:
data_clean2.query('100 < level_1 < 200')

In [ ]:
data_clean2.query('100 < level_1 < 200').iplot(kind='scatter3d', x='chan', y='level_1', mode='markers', z=0, size=6, 
                  title='Phase BL', filename='phase_test', width=1, opacity=0.8, colors='blue', symbol='circle',
                  layout={'scene': {'aspectratio': {'x': 1, 'y': 3, 'z': 0.7}}})

In [ ]:
ploting = data_clean2.query('100 < level_1 < 200').figure(kind='scatter3d', x='chan', y='level_1', mode='markers', z=0, size=6, 
                  title='Phase BL', filename='phase_test', width=1, opacity=0.8, colors='blue', symbol='circle',
                  layout={'scene': {'aspectratio': {'x': 1, 'y': 3, 'z': 0.7}}})

In [ ]:
ploting

In [ ]:
ploting.data[0]['marker']['color'] = 'blue'
ploting.data[0]['marker']['line'] = {'color': 'blue', 'width': 0.5}
ploting.data[0]['marker']['opacity'] = 0.5

In [ ]:
py.iplot(ploting, filename='phase_test')

## Matplotlib

In [ ]:
fig=plt.figure()
ax=fig.gca(projection='3d')

X = np.arange(0, data_clean.shape[1],1)
Y = np.arange(0, data_clean.shape[0],1)

X, Y = np.meshgrid(X,Y)

surf = ax.scatter(X, Y, data_clean, '.', c=data_clean,s=2,lw=0,cmap='winter')

In [ ]:
%matplotlib notebook

In [ ]:
data_clean2.plot(kind='scatter', x='chan', y=0)

In [ ]:
import seaborn as sns

In [ ]:
data_clean2.plot(kind='scatter', x='level_1', y=0)

In [ ]:
corr.head()

In [ ]:
data_ready['noise'] = todegclean.iloc[:,2:].std(axis=1)

In [ ]:
data_ready[['ant1name', 'ant2name', 'noise']]

In [ ]:
corr = data_ready[['ant1name', 'ant2name', 'noise']].pivot_table(index=['ant1name'], columns=['ant2name'])

In [ ]:
corr.columns.levels[1]

In [ ]:
corr2 = pd.DataFrame(corr.values, index=corr.index.values, columns=corr.columns.levels[1].values)

In [ ]:
corr2

In [ ]:
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr2, cmap=cmap,
            square=True, xticklabels=5, yticklabels=5,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)